<a href="https://colab.research.google.com/github/Manikanta2804/hpc_2026_b07_2303A51458/blob/main/HPC_Lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#2303A51458
Batch-7

Section 1: Serial Recursive Fibonacci (Baseline)

In [ ]:
import time

def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

def main(n_value):
    start_time = time.time()
    result = fibonacci(n_value)
    end_time = time.time()

    execution_time = end_time - start_time

    print(f"Fibonacci({n_value}) = {result}")
    print(f"Execution time: {execution_time:.6f} seconds")
n = 15
if __name__ == "__main__":
    main(n)

Fibonacci(15) = 610
Execution time: 0.000119 seconds


Section 2: OpenMP Task-based Fibonacci (Basic Tasking)

In [ ]:
import time
from numba import njit
from concurrent.futures import ThreadPoolExecutor

#serial Fibonacci
@njit
def fib_serial(n):
    if n <= 1:
        return n
    return fib_serial(n - 1) + fib_serial(n - 2)

#Parallel Fibonacci (Task Style)
def fib_parallel(n,threshold=20):
  if n<=1:
    return n
  if n<=threshold:
    return fib_serial(n)

  with ThreadPoolExecutor() as executor:
    future1=executor.submit(fib_parallel,n-1, threshold)
    future2=executor.submit(fib_parallel,n-2, threshold)
    return future1.result()+future2.result

def main():
  n=int(input("Enter the value of n: "))

  #serial Execution
  start=time.time()
  serial_result=fib_serial(n)
  serial_time=time.time() - start

  print("\nSerial Fibonacci: ",serial_result)
  print("Serial Execution Time: ",serial_time)

  #parallel Execution
  start=time.time()
  parallel_result=fib_parallel(n)
  parallel_time=time.time() - start

  print("\nParallel Fibonacci: ",parallel_result)
  print("Parallel Execution Time: ",parallel_time)

if __name__=="__main__":
  main()

Enter the value of n: 15

Serial Fibonacci:  610
Serial Execution Time:  0.07079315185546875

Parallel Fibonacci:  610
Parallel Execution Time:  1.33514404296875e-05


Section 3: Task Creation and Synchronization using taskwait


In [ ]:
import time
from numba import njit
from concurrent.futures import ThreadPoolExecutor

#serial Fibonacci
@njit
def fib_serial(n):
    if n <= 1:
        return n
    return fib_serial(n - 1) + fib_serial(n - 2)

# Parallel Fibonacci With Task Synchronization
def fib_taskwait(n,executor,threshold=20):
  if n<=1:
    return n

  if n<threshold:
    return fib_serial(n)

  #Task creation
  future1 = executor.submit(fib_taskwait, n - 1, executor, threshold)
  future2 = executor.submit(fib_taskwait, n - 2, executor, threshold)

  #taskwait Simulation
  x=future1.result()
  y=future2.result()

  return x+y

def main():
  n=int(input("Enter Value of n: "))
  start=time.time()
  with ThreadPoolExecutor() as executor:
    result=fib_taskwait(n,executor)
  end=time.time()

  print(f"Fibonacci ({n}) = {result}")
  print("Execution Time: ",end-start,"seconds")

if __name__=="__main__":
  main()


Enter Value of n: 15
Fibonacci (15) = 610
Execution Time:  0.07312417030334473 seconds


Section 4: Performance Analysis of OpenMP Tasking


In [ ]:
import time
from concurrent.futures import ProcessPoolExecutor
THRESHOLD = 20
def fib_serial(n):
    if n <= 1:
        return n
    return fib_serial(n - 1) + fib_serial(n - 2)

def fib_task(n):
    if n <= 1:
        return n

    if n < THRESHOLD:
        return fib_serial(n)

    with ProcessPoolExecutor() as executor:
        f1 = executor.submit(fib_task, n - 1)
        f2 = executor.submit(fib_task, n - 2)
        return f1.result() + f2.result()

if __name__ == "__main__":
    test_values = [20, 25, 30, 35]
    print("Task-Based Fibonacci Performance Analysis\n")
    for n in test_values:
        start = time.perf_counter()
        result = fib_task(n)
        end = time.perf_counter()

        print(f"Fibonacci({n}) = {result}")
        print(f"Execution Time: {end - start:.6f} seconds\n")

Task-Based Fibonacci Performance Analysis

Fibonacci(20) = 6765
Execution Time: 0.025213 seconds

Fibonacci(25) = 75025
Execution Time: 0.337935 seconds

Fibonacci(30) = 832040
Execution Time: 5.326967 seconds

Fibonacci(35) = 9227465
Execution Time: 63.774130 seconds



In [ ]:
import time
from concurrent.futures import ProcessPoolExecutor

def chunk_sum(chunk):
    return sum(chunk)

if __name__ == "__main__":
    arr = list(range(1, 1_000_001))
    num_workers = 4
    chunk_size = len(arr) // num_workers

    chunks = [
        arr[i:i + chunk_size]
        for i in range(0, len(arr), chunk_size)
    ]

    start = time.perf_counter()

    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        partial_sums = executor.map(chunk_sum, chunks)

    total_sum = sum(partial_sums)
    end = time.perf_counter()

    print("Loop-Based Parallel Sum")
    print("Sum =", total_sum)
    print("Execution Time:", end - start)

Loop-Based Parallel Sum
Sum = 500000500000
Execution Time: 0.39496521500041126
